In [36]:
import pandas as pd
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine
# Load .env enviroment variables into the notebook
load_dotenv()
# Get the postgres connection information from os file. 

DB_HOST = os.getenv('DB_HOST')
DB_NAME = os.getenv('DB_NAME')
DB_USER = os.getenv('DB_USER')
DB_PASS = os.getenv('DB_PASS')

In [37]:

real_acct_df = pd.read_csv('../real_estate_data/real_acct_owner/real_acct.txt', sep='\t',lineterminator='\r',header=0,encoding='latin1',low_memory=False)

In [38]:
columns = real_acct_df.columns
columns

Index(['acct', 'yr', 'mailto', 'mail_addr_1', 'mail_addr_2', 'mail_city',
       'mail_state', 'mail_zip', 'mail_country', 'undeliverable', 'str_pfx',
       'str_num', 'str_num_sfx', 'str', 'str_sfx', 'str_sfx_dir', 'str_unit',
       'site_addr_1', 'site_addr_2', 'site_addr_3', 'state_class',
       'school_dist', 'map_facet', 'key_map', 'Neighborhood_Code',
       'Neighborhood_Grp', 'Market_Area_1', 'Market_Area_1_Dscr',
       'Market_Area_2', 'Market_Area_2_Dscr', 'econ_area', 'econ_bld_class',
       'center_code', 'yr_impr', 'yr_annexed', 'splt_dt', 'dsc_cd', 'nxt_bld',
       'bld_ar', 'land_ar', 'acreage', 'Cap_acct', 'shared_cad', 'land_val',
       'bld_val', 'x_features_val', 'ag_val', 'assessed_val', 'tot_appr_val',
       'tot_mkt_val', 'prior_land_val', 'prior_bld_val',
       'prior_x_features_val', 'prior_ag_val', 'prior_tot_appr_val',
       'prior_tot_mkt_val', 'new_construction_val', 'tot_rcn_val',
       'value_status', 'noticed', 'notice_dt', 'protested', 'certif

In [39]:
# new_cols = ['acct', 'str_pfx', 'str_num',
#        'str_num_sfx', 'str', 'str_sfx', 'str_sfx_dir', 'str_unit',
#        'site_addr_1', 'site_addr_2', 'site_addr_3','bld_val', 'x_features_val', 'ag_val', 
#        'assessed_val', 'tot_appr_val',
#        'tot_mkt_val', 'prior_land_val', 'prior_bld_val',
#        'prior_x_features_val', 'prior_ag_val', 'prior_tot_appr_val',
#        'prior_tot_mkt_val', 'new_construction_val', 'tot_rcn_val',
#        'value_status', 'certified_date']

In [40]:
new_cols = ['acct','str_num','str_num_sfx','str','str_sfx','str_sfx_dir','str_unit','site_addr_1','site_addr_2',
'site_addr_3','bld_ar','land_ar','tot_appr_val','tot_mkt_val']

In [41]:
# remove the columns that we don't need
real_acct_df = real_acct_df[new_cols]
real_acct_df.head()

,acct,str_num,str_num_sfx,str,str_sfx,str_sfx_dir,str_unit,site_addr_1,site_addr_2,site_addr_3,bld_ar,land_ar,tot_appr_val,tot_mkt_val
0,\n0010010000013,0.0,,COMMERCE,ST,,,0 COMMERCE ST,HOUSTON,77002,0.0,44431.0,0.0,0.0
1,\n0010020000001,907.0,,COMMERCE,ST,,,907 COMMERCE ST,HOUSTON,77002,0.0,5001.0,309120.0,309120.0
2,\n0010020000003,0.0,,COMMERCE,ST,,,0 COMMERCE ST,HOUSTON,77002,0.0,18121.0,889398.0,889398.0
3,\n0010020000004,0.0,,COMMERCE,ST,,,0 COMMERCE ST,HOUSTON,77002,0.0,9061.0,444699.0,444699.0
4,\n0010020000013,921.0,,COMMERCE,ST,,,921 COMMERCE ST,HOUSTON,77002,0.0,3001.0,0.0,0.0


In [42]:
# remove the \n from the account column
def clean(x):
    x = x.replace('\n','')
    return x

In [43]:
real_acct_df['acct'] = real_acct_df['acct'].apply(clean)

In [44]:
real_acct_df.head(2)

,acct,str_num,str_num_sfx,str,str_sfx,str_sfx_dir,str_unit,site_addr_1,site_addr_2,site_addr_3,bld_ar,land_ar,tot_appr_val,tot_mkt_val
0,0010010000013,0.0,,COMMERCE,ST,,,0 COMMERCE ST,HOUSTON,77002,0.0,44431.0,0.0,0.0
1,0010020000001,907.0,,COMMERCE,ST,,,907 COMMERCE ST,HOUSTON,77002,0.0,5001.0,309120.0,309120.0


In [45]:
# load 5 million rows to local database
engine = create_engine(f'postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:5432/Houston_real_estate')
# load df to database
real_acct_df.to_sql('real_acct',engine,index=False,if_exists='replace')

In [46]:
real_acct_df.dtypes

acct             object
str_num         float64
str_num_sfx      object
str              object
str_sfx          object
str_sfx_dir      object
str_unit         object
site_addr_1      object
site_addr_2      object
site_addr_3      object
bld_ar          float64
land_ar         float64
tot_appr_val    float64
tot_mkt_val     float64
dtype: object